In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import PIL.Image as Image

In [7]:
def load_dataframe(path):
    df = pd.read_csv(path)
    df = df.dropna()
    df = df.reset_index(drop=True)
    return df

train_path = '/home/jon/development/university/sis/cv_api/data/sign_mnist/sign_mnist_train.csv'
test_path = '/home/jon/development/university/sis/cv_api/data/sign_mnist/sign_mnist_test.csv'

train_dataframe = load_dataframe(train_path)
test_dataframe = load_dataframe(test_path)


In [8]:
def form_datasets(labels, images):
    if len(labels) != len(images):
        raise Exception('The number of labels and images are not the same.')

    output_labels = []
    output_images = []

    for x in range(len(labels)):
        label = labels[x]
        image = images[x]
        image = image.reshape((28, 28))

        output_labels.append(label)
        output_images.append(image)

    return np.array(output_labels).astype(float), np.array(output_images).astype(float)


train_labels, train_images = form_datasets(train_dataframe['label'].to_numpy(), train_dataframe.drop('label', axis=1).to_numpy())
test_labels, test_images = form_datasets(test_dataframe['label'].to_numpy(), test_dataframe.drop('label', axis=1).to_numpy())

train_images = np.expand_dims(train_images, axis=3) # Expanding Dimension for Convolutional Layer
test_images = np.expand_dims(test_images, axis=3) # Expanding Dimension for Convolutional Layer

In [9]:
def getImageGenerators(training_labels, training_images, test_labels, test_images):
    datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        horizontal_flip=True,
        zoom_range=0.2,
        fill_mode='nearest'
    )

    test_datagen = ImageDataGenerator(
        rescale=1./255
    )

    train_generator = datagen.flow(x=training_images, y=training_labels, batch_size=32)
    test_generator = test_datagen.flow(x=test_images, y=test_labels, batch_size=32)

    return train_generator, test_generator

train_generator, test_generator = getImageGenerators(train_labels, train_images, test_labels, test_images)

In [10]:
def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPool2D((2, 2)),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPool2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(26, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

model = build_model()

epochs = 20
history = model.fit(train_generator, epochs=epochs, validation_data=test_generator)

model.summary()

Epoch 1/20


2023-08-29 21:47:43.821891: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-08-29 21:47:44.337233: W tensorflow/compiler/xla/service/gpu/llvm_gpu_backend/gpu_backend_lib.cc:274] libdevice is required by this HLO module but was not found at ./libdevice.10.bc
2023-08-29 21:47:44.337810: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:362 : INTERNAL: libdevice not found at ./libdevice.10.bc
2023-08-29 21:47:44.337832: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:GPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INTERNAL: libdevice not found at ./libdevice.10.bc
	 [[{{node StatefulPartitionedCall_6}}]]


InternalError: Graph execution error:

Detected at node 'StatefulPartitionedCall_6' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "/home/jon/miniconda3/envs/sis_cv_api/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/jon/miniconda3/envs/sis_cv_api/lib/python3.11/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/jon/miniconda3/envs/sis_cv_api/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 736, in start
      self.io_loop.start()
    File "/home/jon/miniconda3/envs/sis_cv_api/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/home/jon/miniconda3/envs/sis_cv_api/lib/python3.11/asyncio/base_events.py", line 607, in run_forever
      self._run_once()
    File "/home

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.plot(epochs_range, accuracy, 'g', label='Training Accuracy')
plt.plot(epochs_range, val_accuracy, 'b', label='Validation Accuracy')
plt.legend()
plt.figure()
plt.title('Training / Validation Accuracy')

plt.plot(epochs_range, loss, 'r', label='Training Loss')
plt.plot(epochs_range, val_loss, 'y', label='Validation Loss')
plt.title('Training / Validation Loss')
plt.legend()
plt.figure()

: 